## $R^{2}$ mistake
Una delle metriche per misurare le performance della curva di regressione, e quindi anche della previsione su serie temporali è $R^{2}$.
<br>
Prediamo la definizione direttamente da [Wikipedia](https://it.wikipedia.org/wiki/Coefficiente_di_determinazione):
<br>
*In statistica, il coefficiente di determinazione, (più comunemente $R^{2}$), è una proporzione tra la variabilità dei dati e la correttezza del modello statistico utilizzato. Esso misura la frazione della varianza della variabile dipendente espressa dalla regressione. Non esiste una definizione concordata di $R^{2}$. Nelle regressioni lineari semplici esso è semplicemente il quadrato del coefficiente di correlazione.*
<br>

In questo notebook vedremo come non sempre i valori alti di $R^{2}$ siano indice di buone previsioni ma semplicemente di un modello che non impara.
<br>
Per fare ciò **"ammazzeremo le mosche col bazooka"**, ovvero utilizzeremo una rete neurale MLP per fare previsione su Airpassengers.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
airpassengers = pd.read_csv("airpassenger.csv")
airpassengers.columns = ["Time","Passengers"]
airpassengers.head()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers", xlabel="Time", ylabel="Passengers")
plt.show()

Per prima cosa bisogna preparare i dati per la rete neurale. Non divideremo in train e test perchè in questo caso non ci interessa la previsione in se, quindi basta il lavoro sul train.
<br>
Le reti neurali lavorano con i tensori, le serie temporali sono tensori a tre dimensioni **[samples, timestep, features]**, nel nostro caso essendo univariata la features sarà pari ad uno.
<br>
Dividiamo la nostra serie in un N samples di dimensione 12 (timestep) e per ognuno di questi prevediamo il tredicesimo valore. Quindi ci serve un anno per predire un mese.
<br>
**Esempio:**
[1,2,3,4,5,6,7,8,9,10] a timestep 3 diventa:
* [1,2,3] [4]
* [2,3,4] [5]
* [3,4,5] [6]
* ...
<br>

Per prima cosa estraiamo dal DataFrame la cofeatures con i valori creando un array numpy

In [ ]:
airpassengers_array = airpassengers["Passengers"].values

In [ ]:
def split_sequence(sequence, n_steps=12):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
X, y = split_sequence(airpassengers_array)

Vediamo cosa ci restituisce la funzione. Avremo 132 vettori da 12 elementi in X e un vettore da 132 elementi in y.

In [ ]:
for i in range(len(X)):
    print(X[i], y[i])

Per il nostro esperimento proviamo a usare 3 mesi come timestep e predire il quarto.
<br>
Successivamente creiamo un Multilayer Perceptron abbastanza semplice con tf.keras

In [ ]:
X, y = split_sequence(airpassengers_array, n_steps=3)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(12, activation='relu', input_dim=3))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))
model.summary()

Come possiamo vedere il modello è abbastanza semplice con due livelli nascosti ma ha già 161 parametri da allenare.
<br>
Breve parentesi, cosa sono i parametri? Sono le connessioni tra i neuroni.
* noi abbiamo 3 "nodi" in input e 12 nel primo strato nascosto, quindi 3 x 12 = 36 connessioni + il bias, uno per ogni nodo hidden quindi altri 12 che sommati ai 36 fa 48.
* Nel secondo strato 12 x 8 = 96 + 8 = 104
* Infine un solo nodo di output trattandosi di regressione, quindi 8 archi entranti dallo strato hidden più quello del bias fanno 9.
<br>

**48 + 104 + 9 = 161**

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X, y, epochs = 200)

Ora che abbiamo allenato il modello facciamo la previsione sullo stesso, è inusuale ma visto lo scopo del test và bene.

In [ ]:
yhat = model.predict(X)

Calcoliamo $R^{2}$ tra i valori reali e i predetti, come vedremo sarà molto alto.

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y, yhat)

Un $R^{2}$ del 90% si direbbe una buona previsione ma non lo è e per questo ci basta fare una veloce analisi grafica.
<br>

Come possiamo vedere il modello non ha imparato nulla ma ha solo **"copiato"** il valore al perido $t-1$ come previsione per $t$. Questo si verifica solitamente quando ci sono pochi valori e la rete neurale non riesce ad **apprendere** ovvero a tarare i pesi in modo da riprodurre uno schema.

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(len(y)), y, color='tab:blue')
plt.plot(range(len(yhat)), yhat, color='tab:red')
plt.gca().set(title="Airpassengers", xlabel="Time", ylabel="Passengers")
plt.show()

Se volte provare un altro esperimento vi consiglio di:
* utilizzare *split_sequence* con 12 steps come di default
* allenare lo stesso modello o un modello simile
* fare la previsione sul train
<br>

Questa soluzione dovrebbe andare in **overfitting** (e salire all'aumentare delle epoche di apprendimento)

**N.B. se volete approfondire il time series forecasting usando il MLP (sconsiglio questo tipo di rete neurale per il problema da affrontare) in Keras potete consultare [questo link](https://machinelearningmastery.com/time-series-prediction-with-deep-learning-in-python-with-keras/)**